<a href="https://colab.research.google.com/github/hanbitlee/summer_2021/blob/main/gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dgl

In [ ]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_u(u='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

In [ ]:
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
    
    def forward(self, g, feature):
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [ ]:
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset, citation_graph
def load_data():
    dataset = citation_graph()
    g = dataset[0]
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    test_mask = g.ndata['test_mask']
    return g, features, labels, train_mask, test_mask

In [ ]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(300):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch 00000 | Loss 0.2751 | Test Acc 0.4950 | Time(s) nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00001 | Loss 10.5335 | Test Acc 0.4850 | Time(s) nan
Epoch 00002 | Loss 4.0080 | Test Acc 0.4960 | Time(s) nan
Epoch 00003 | Loss 1.8973 | Test Acc 0.4530 | Time(s) 0.0605
Epoch 00004 | Loss 3.3163 | Test Acc 0.5120 | Time(s) 0.0717
Epoch 00005 | Loss 1.9648 | Test Acc 0.5670 | Time(s) 0.0679
Epoch 00006 | Loss 2.1475 | Test Acc 0.5860 | Time(s) 0.0659
Epoch 00007 | Loss 2.2179 | Test Acc 0.5890 | Time(s) 0.0653
Epoch 00008 | Loss 2.0054 | Test Acc 0.5820 | Time(s) 0.0648
Epoch 00009 | Loss 1.6212 | Test Acc 0.5880 | Time(s) 0.0645
Epoch 00010 | Loss 1.1924 | Test Acc 0.5590 | Time(s) 0.0642
Epoch 00011 | Loss 0.8261 | Test Acc 0.5470 | Time(s) 0.0639
Epoch 00012 | Loss 0.5798 | Test Acc 0.5640 | Time(s) 0.0639
Epoch 00013 | Loss 0.5344 | Test Acc 0.5610 | Time(s) 0.0647
Epoch 00014 | Loss 0.6599 | Test Acc 0.5550 | Time(s) 0.0644
Epoch 00015 | Loss 0.8698 | Test Acc 0.5540 | Time(s) 0.0641
Epoch 00016 | Loss 0.9752 | Test Acc 0.5530 | Time(s) 0.0638
Epoch 00017 | Loss 0.9553 | T